In [59]:
from collections import namedtuple
from sklearn.model_selection import train_test_split
import pandas as pd
import zipfile
from cementStrength.constants import *
from cementStrength.utils import read_yaml, create_directories
import os
import urllib.request as request
from zipfile import ZipFile
import tarfile
from cementStrength import logger
os.chdir("../")
import yaml

In [ ]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir",
    "ingested_data_train_dir",
    "ingested_data_test_dir"])

In [69]:
with open(CONFIG_FILE_PATH) as yaml_file:
        content = yaml.safe_load(yaml_file)

In [70]:
from box import ConfigBox
x = ConfigBox(content)

In [71]:
x

ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls', 'local_data_file': 'artifacts/data_ingestion/data.xls', 'unzip_dir': 'artifacts/data_ingestion', 'ingested_data_train_dir': 'artifacts/data_ingestion/train', 'ingested_data_test_dir': 'artifacts/data_ingestion/test'}})

In [73]:
x.data_ingestion.root_dir

'artifacts/data_ingestion'

In [77]:



class ConfigurationManager:
    def __init__(self, 
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        create_directories([config.ingested_data_train_dir, config.ingested_data_test_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            ingested_data_train_dir = config.ingested_data_train_dir,
            ingested_data_test_dir= config.ingested_data_test_dir
        )
        return data_ingestion_config



In [82]:
import logging


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        logger.info("downloading data file")
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename = self.config.local_data_file
            )
        logger.info("file downloaded")

    # def unzip_file(self):
    #     logger.info("extracting zip file")
    #     with ZipFile(file=self.config.local_data_file) as zf:
    #         zf.extractall(self.config.unzip_dir)
    #     logger.info("file extraction done")


    def split_data_as_train_test(self):
        downloaded_file_path = self.config.local_data_file
        train_filename = "train.csv"
        test_filename = "test.csv"
        train_file_path = os.path.join(self.config.ingested_data_train_dir, train_filename)
        test_file_path = os.path.join(self.config.ingested_data_test_dir, test_filename)

        logger.info("reading excel file into pandas dataframe")
        cement_strength_df = pd.read_excel(downloaded_file_path)
        X = cement_strength_df.iloc[:,:-1]
        y = cement_strength_df.iloc[:,-1]
        logger.info("splitting df in train and test")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

        if X_train is not None:
            logger.info("saving X_train....")
            X_train.to_csv(train_file_path, index=False)
            logger.info("saved X_train")

        if X_test is not None:
            logger.info("saving X_test....")
            X_test.to_csv(test_file_path, index=False)
            logger.info("saved X_test")


In [83]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.split_data_as_train_test()
except Exception as e:
    raise e

[2022-10-06 20:27:39,032: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-10-06 20:27:39,040: INFO: common]: yaml file: params.yaml loaded successfully
[2022-10-06 20:27:39,044: INFO: common]: created directory at: artifacts
[2022-10-06 20:27:39,052: INFO: common]: created directory at: artifacts/data_ingestion
[2022-10-06 20:27:39,060: INFO: common]: created directory at: artifacts/data_ingestion/train
[2022-10-06 20:27:39,065: INFO: common]: created directory at: artifacts/data_ingestion/test
[2022-10-06 20:27:39,069: INFO: 611451049]: downloading data file
[2022-10-06 20:27:42,303: INFO: 611451049]: file downloaded
[2022-10-06 20:27:42,307: INFO: 611451049]: reading excel file into pandas dataframe
[2022-10-06 20:27:42,486: INFO: 611451049]: splitting df in train and test
[2022-10-06 20:27:42,491: INFO: 611451049]: saving X_train....
[2022-10-06 20:27:42,503: INFO: 611451049]: saved X_train
[2022-10-06 20:27:42,505: INFO: 611451049]: saving X_test....
[2022-1

In [84]:
import pandas 